<a href="https://colab.research.google.com/github/carooobregon/Hateful-Memes/blob/main/TextClassificationFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/t-davidson/hate-speech-and-offensive-language.git

Cloning into 'hate-speech-and-offensive-language'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 41 (delta 4), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (41/41), done.


In [ ]:
import pandas as pd

In [ ]:
dataset=pd.read_csv("/content/hate-speech-and-offensive-language/data/labeled_data.csv")
dataset

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
# Visualizando si hay missing values
X_complete = pd.DataFrame(dataset.shape[0] - dataset.isnull().sum(), columns=['CompleteSamples'])
X_complete

,CompleteSamples
Unnamed: 0,24783
count,24783
hate_speech,24783
offensive_language,24783
neither,24783
class,24783
tweet,24783


In [ ]:
#the class label indicate  0 - hate speech 1 - offensive language 2 - neither
dataset2=dataset.loc[:,["tweet","class"]]
dataset2

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
24779,"you've gone and broke the wrong heart baby, an...",2
24780,young buck wanna eat!!.. dat nigguh like I ain...,1
24781,youu got wild bitches tellin you lies,1


In [ ]:
dataset2.iloc[0,0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [ ]:
#eliminamos los  urls,usuarios, el RT, los signos de exclamación,dos puntos, &user,# y ~~.
dataset2['tweet']= dataset2['tweet'].str.replace('http\S+|www.\S+', '', case=False)
dataset2['tweet']= dataset2['tweet'].str.replace('@[0-9A-Za-z-_]*', '', case=False)
dataset2['tweet']= dataset2['tweet'].str.replace('RT', '', case=False)
dataset2['tweet']= dataset2['tweet'].str.replace('!|:|&[0-9A-Za-z-_]*|#[0-9A-Za-z-_]*|~~[0-9A-Za-z-_]*', '', case=False)

In [ ]:
dataset2.iloc[0,0]

"woman shouldn't complain cleaning house. ; man always take trash out..."

In [ ]:
#Utilizando NLTK, remueve todas las stopwords aexcepción de aquellas que indiquennegación (i.e. no, not)
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# array de las palabras que queremos conservar de stopwords
stopwords2=set(nltk.corpus.stopwords.words('english'))
negativewords=set(('against','any','no','nor','not','t',"don't",'ain',"aren't","couldn't","didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't"))
stopwordsfinal = stopwords2.difference(negativewords)

In [ ]:
#Eliminamos las palabras que esten en los stopwords a excepción de las palabras negativas
dataset2['tweet'] = dataset2['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stopwordsfinal)]))
dataset2

,tweet,class
0,woman shouldn't complain cleaning house. ; man...,2
1,boy dats cold...tyga dwn bad cuffin dat hoe 1s...,1
2,Dawg ever fuck bitch sta cry? confused shit,1
3,look like tranny,1
4,shit hear might true might faker bitch told ya ;,1
...,...,...
24778,"you's muthaf***in lie ; right TL trash ;. Now,...",1
24779,"gone broke wrong hea baby, drove redneck crazy",2
24780,young buck wanna eat.. dat nigguh like aint fu...,1
24781,youu got wild bitches tellin lies,1


In [ ]:
#cantidad de ejemplos por clases
print(len(dataset2[dataset2["class"]==0]))
print(len(dataset2[dataset2["class"]==1]))
print(len(dataset2[dataset2["class"]==2]))

1430
19190
4163


In [ ]:
#Al ser distintas cantidades de ejemplos por clases solo seleccionamos 1430 por cada clase
dataset2[dataset2["class"]==1].iloc[0:2733,:]

,tweet,class
1,boy dats cold...tyga dwn bad cuffin dat hoe 1s...,1
2,Dawg ever fuck bitch sta cry? confused shit,1
3,look like tranny,1
4,shit hear might true might faker bitch told ya ;,1
5,""" shit blows me..claim faithful somebody still...",1
...,...,...
3544,better feel love... don't even show hoes love lol,1
3545,lmao denote got bitch right love,1
3546,want somewhat collage .. Money jewels caiers e...,1
3547,nicca anxious hear shit g,1


In [ ]:
#mezclamos las instancias de la clase 1 y extraemos 2733, luego las concatenamos con las 1430 de la clase 0 y volvemos a mezclarlas y hacemos una unica clase.
from sklearn.utils import shuffle
class1=dataset2[dataset2["class"]==1]
#print(class1)
class1sh=shuffle(class1)
class1sh_subset=class1sh.iloc[0:2733,:]
#print(class1sh_subset)
class0=dataset2[dataset2["class"]==0]

In [ ]:
dataset3=pd.concat([class0,class1sh_subset])
dataset3

,tweet,class
85,""" queer"" gaywad",0
89,""" alsarabsss"" hes beaner smh tell hes mexican",0
110,""" fucking gay, blacklisted hoe"" Holding anyway",0
184,""" LMFAOOOO HATE BLACK PEOPLE there's black peo...",0
202,""" ""At least I'm not nigger"" Lmfao",0
...,...,...
14541,wanna skinny ; fit. SIKE Thick bitches got goi...,1
19429,People like bae real quick like bitch don't pl...,1
78,""" 0 rings 0 mvps 0 bitches lol",1
10555,"hope trip fall pussy, tunechi",1


In [ ]:
dataset3sh=shuffle(dataset3)
dataset3sh

,tweet,class
10095,guarantee 9 10 y'all's bitches follow me.,1
9393,George michael fag,0
17540,hoe ass nigga never change,1
22120,niggah know wassup,0
4730,teabagger scum wants threaten people penis ext...,0
...,...,...
4021,ya 5 games retard?,1
11822,"Instead calling hoes, girls suppoing other. Id...",1
4562,one rawest niccas doin dis shit,1
16324,sta crying stalking faggot. want laff,0


In [ ]:
dataset3sh['class']= dataset3sh['class'].replace(1,0)
dataset3sh

,tweet,class
10095,guarantee 9 10 y'all's bitches follow me.,0
9393,George michael fag,0
17540,hoe ass nigga never change,0
22120,niggah know wassup,0
4730,teabagger scum wants threaten people penis ext...,0
...,...,...
4021,ya 5 games retard?,0
11822,"Instead calling hoes, girls suppoing other. Id...",0
4562,one rawest niccas doin dis shit,0
16324,sta crying stalking faggot. want laff,0


In [ ]:
# cambiamos la clase 2 por el num 1
class2=dataset2[dataset2["class"]==2]
class2['class']= class2['class'].replace(2,1)
class2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,tweet,class
0,woman shouldn't complain cleaning house. ; man...,1
40,""" momma said no pussy cats inside doghouse """,1
63,""" -SimplyAddictedToGuys woof woof hot scally lad",1
66,""" woof woof hot soles",1
67,""" Lemmie eat Oreo ; dishes."" One oreo? Lol",1
...,...,...
24736,yaya ho.. cute avi tho no idea sleep ;,1
24737,yea 's new friend.. friends kno they're allowe...,1
24767,"know say, early bird gets worm. *puts gummy wo...",1
24779,"gone broke wrong hea baby, drove redneck crazy",1


In [ ]:
# concatenamos clase 0 y 1, ahora 0 indica (hate speech y offensive language) y label 1 indica neither
dataset_final=pd.concat([dataset3sh,class2])
dataset_final

,tweet,class
10095,guarantee 9 10 y'all's bitches follow me.,0
9393,George michael fag,0
17540,hoe ass nigga never change,0
22120,niggah know wassup,0
4730,teabagger scum wants threaten people penis ext...,0
...,...,...
24736,yaya ho.. cute avi tho no idea sleep ;,1
24737,yea 's new friend.. friends kno they're allowe...,1
24767,"know say, early bird gets worm. *puts gummy wo...",1
24779,"gone broke wrong hea baby, drove redneck crazy",1


In [ ]:
#hacemos el ultimo shuffle de todo el dataset
dataset_finalsh=shuffle(dataset_final)
dataset_finalsh

,tweet,class
19814,"""There's one way skin monkey"" --Barack ""Bushme...",1
84,""" Ok, sis."" She'd rather broke bitch? *shrugs*...",0
4730,teabagger scum wants threaten people penis ext...,0
19040,trash,1
679,Matthew 626 Look birds air.. yet heavenly Fath...,1
...,...,...
717,Happy Fall Pumpkin Spice Oreos,1
13731,Oscar Taveras went yardie 2nd career MLB bat. ...,1
9478,Golden Oreos best,1
23678,bitches dont eat meat likely develop mental il...,0


In [ ]:
#Se extrae el vocabulario de todos los tweets
from keras.preprocessing.text import Tokenizer
Obj=Tokenizer()
Obj.fit_on_texts(dataset_finalsh['tweet'])
Obj.word_index

{'bitch': 1,
 'trash': 2,
 'like': 3,
 'bitches': 4,
 "i'm": 5,
 'not': 6,
 "don't": 7,
 'no': 8,
 'ass': 9,
 'u': 10,
 'hoes': 11,
 'get': 12,
 'pussy': 13,
 'got': 14,
 'hoe': 15,
 'lol': 16,
 'bird': 17,
 'fuck': 18,
 'nigga': 19,
 'charlie': 20,
 'shit': 21,
 'one': 22,
 'yellow': 23,
 'faggot': 24,
 'white': 25,
 'know': 26,
 'go': 27,
 'man': 28,
 'love': 29,
 'people': 30,
 "ain't": 31,
 'good': 32,
 'birds': 33,
 'want': 34,
 'fucking': 35,
 'niggas': 36,
 'look': 37,
 "can't": 38,
 'make': 39,
 'ghetto': 40,
 'say': 41,
 'back': 42,
 'would': 43,
 'still': 44,
 'hate': 45,
 "that's": 46,
 'time': 47,
 'yo': 48,
 'think': 49,
 'day': 50,
 'even': 51,
 'really': 52,
 'colored': 53,
 'see': 54,
 'it': 55,
 'monkey': 56,
 'yankees': 57,
 'never': 58,
 'you': 59,
 'right': 60,
 'new': 61,
 'nigger': 62,
 'need': 63,
 'said': 64,
 '2': 65,
 'im': 66,
 "y'all": 67,
 'bad': 68,
 'black': 69,
 'real': 70,
 'take': 71,
 'game': 72,
 'ho': 73,
 'lmao': 74,
 'da': 75,
 'girl': 76,
 'going

In [ ]:
# se obtiene la secuencia de cada tweet conforme al indice obtenido
from keras.preprocessing.sequence import pad_sequences
Obj2=pad_sequences(Obj.texts_to_sequences(dataset_finalsh['tweet']))
Obj2

array([[    0,     0,     0, ...,  2765,  5538,   340],
       [    0,     0,     0, ...,  2177,   105,  1274],
       [    0,     0,     0, ...,  5539,  3656,   581],
       ...,
       [    0,     0,     0, ...,  2805,   212,    94],
       [    0,     0,     0, ..., 12380,  1780, 12381],
       [    0,     0,     0, ...,  3964,  4967,     1]], dtype=int32)

In [ ]:
#Descargargamos el zip de internet de glove6B desde colab
!wget 'http://nlp.stanford.edu/data/glove.6B.zip'


--2021-11-28 23:21:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-28 23:21:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-28 23:21:52--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
#nombre de la carpeta donde se guardaran los archivos descomprimidos
! mkdir glove

mkdir: cannot create directory ‘glove’: File exists


In [ ]:
# Descomprimirlo en una carpeta creada
!unzip glove.6B.zip -d glove

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [ ]:
#hacemos un respaldo en nuestro drive
!cp -r glove /content/drive/Shareddrives/ProyectoTecnologíasEmergentes/Glove

In [ ]:
#creamos un diccionario donde se almacenaran la representación númerica del vocabulario de glove
import numpy as np
embeddings_dict = {}
glove100d=open('/content/glove/glove.6B.100d.txt','r', encoding="utf-8")
for line in glove100d:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector

In [ ]:
#creamos una matriz donde se almacenara la representación numérica segun el vocabulario de glove, del vocabulario de los tweets
embedding_matrix_f=np.zeros(((len(Obj.word_index))+1,100))
c=1
for x in Obj.word_index.keys():
  if x in embeddings_dict:
    embedding_matrix_f[c]=embeddings_dict[x]
  c=c+1
embedding_matrix_f

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33096999,  0.082649  ,  0.66940999, ...,  0.14552   ,
         0.096472  , -0.28279001],
       [-0.23075999,  0.4707    ,  0.76068002, ...,  0.38279   ,
         0.95424998,  0.42825001],
       ...,
       [ 0.048362  ,  0.3637    , -0.40459999, ..., -0.91749001,
         0.93717998, -0.10971   ],
       [ 0.23713   , -0.071172  ,  0.29624999, ..., -0.87325001,
         0.21844999, -1.01269996],
       [-0.34344   ,  0.54587001,  0.42023   , ..., -0.63700002,
        -0.30684   ,  0.64159   ]])

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.metrics import binary_crossentropy

In [ ]:
#creamos un modelo secuencial con las capas de Embedding, LSTM, sigmoid
embedding_2 = Embedding(embedding_matrix_f.shape[0],100,weights=[embedding_matrix_f])
model = Sequential()
model.add(embedding_2)
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1238300   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,318,801
Trainable params: 1,318,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
y=dataset_finalsh['class']
y=np.array(y).astype(int)

In [ ]:
#Entrena el modelo por un numero determinado de epocas 
l=model.fit(Obj2,y,epochs=10,validation_split=0.2)

Epoch 1/10
209/209 [==============================] - 8s 41ms/step - loss: 0.0071 - accuracy: 0.9973 - val_loss: 0.5260 - val_accuracy: 0.8818
Epoch 2/10
209/209 [==============================] - 8s 40ms/step - loss: 0.0055 - accuracy: 0.9980 - val_loss: 0.5575 - val_accuracy: 0.9010
Epoch 3/10
209/209 [==============================] - 8s 40ms/step - loss: 0.0042 - accuracy: 0.9985 - val_loss: 0.5876 - val_accuracy: 0.8980
Epoch 4/10
209/209 [==============================] - 8s 40ms/step - loss: 0.0040 - accuracy: 0.9983 - val_loss: 0.6194 - val_accuracy: 0.8974
Epoch 5/10
209/209 [==============================] - 8s 40ms/step - loss: 0.0046 - accuracy: 0.9980 - val_loss: 0.6757 - val_accuracy: 0.8938
Epoch 6/10
209/209 [==============================] - 8s 41ms/step - loss: 0.0098 - accuracy: 0.9970 - val_loss: 0.4820 - val_accuracy: 0.8884
Epoch 7/10
209/209 [==============================] - 8s 40ms/step - loss: 0.0069 - accuracy: 0.9977 - val_loss: 0.5375 - val_accuracy: 0.9046

In [ ]:
predictionmatrix=np.zeros((Obj2.shape))

In [ ]:
#Para realizar la predicción de nuevo texto ( en este caso las extracciones de texto de las imagenes guardadas en un .csv)
#Primero leemos el csv y hacemos el padding de cada texto de acuerdo con los indices de los tweets y los almacenamos en el array de predictionmatrix

In [ ]:
predictiondataset=pd.read_csv("/content/drive/Shareddrives/ProyectoTecnologíasEmergentes/Datasets/test_images.csv")

In [ ]:
predictiondataset

,Unnamed: 0,Memes,Transcription
0,0,01247.png,vou CaI'he racist ifthere iS H1O other race
1,1,01526.png,[Dut the qoney iT1 a hag. nowl that will he SO.15
2,2,01476.png,II confitleut IIL IIIV OwI skit
3,3,01469.png,wliem voL [UIIC h thne I1LuISLr kucl s0 hard t...
4,4,01735.png,Spending time with Dad
5,5,01578.png,sea monkeys
6,6,01674.png,went On a dlateIling dlate
7,7,01756.png,she iS a fastrumner
8,8,01569.png,when VOu can'( dlecicle t0 pre heat the ovemor...
9,9,01568.png,tlo you think a terroristhas or coult iufiltra...


In [ ]:
ObjPred=pad_sequences(Obj.texts_to_sequences(predictiondataset['Transcription']), maxlen=Obj2.shape[1])

In [ ]:
t=0
for i in ObjPred:
  predictionmatrix[t]=i
  t=t+1

In [ ]:
output_array=model.predict(predictionmatrix)

In [ ]:
final_prediction=[]
for i in output_array:
  if i>0.5:
    final_prediction.append("neutral")
  else:
    final_prediction.append("ofessive-hate")

In [ ]:
final_prediction[0:len(predictiondataset)]

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ofessive-hate',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ofessive-hate',
 'neutral',
 'neutral',
 'neutral',
 'neutral']

In [ ]:
predictiondataset['Classification']=final_prediction[0:len(predictiondataset)]
predictiondataset



,Unnamed: 0,Memes,Transcription,Classification
0,0,01247.png,vou CaI'he racist ifthere iS H1O other race,neutral
1,1,01526.png,[Dut the qoney iT1 a hag. nowl that will he SO.15,neutral
2,2,01476.png,II confitleut IIL IIIV OwI skit,neutral
3,3,01469.png,wliem voL [UIIC h thne I1LuISLr kucl s0 hard t...,neutral
4,4,01735.png,Spending time with Dad,neutral
5,5,01578.png,sea monkeys,neutral
6,6,01674.png,went On a dlateIling dlate,neutral
7,7,01756.png,she iS a fastrumner,neutral
8,8,01569.png,when VOu can'( dlecicle t0 pre heat the ovemor...,neutral
9,9,01568.png,tlo you think a terroristhas or coult iufiltra...,neutral


In [ ]:
#exportar df a csv
predictiondataset.to_csv('text_classification.csv')
!cp  text_classification.csv /content/drive/Shareddrives/ProyectoTecnologíasEmergentes/Results